In [4]:
# #Imports
# #Dataprep
# import matplotlib.pyplot as plt
# import numpy as np
# from nilearn import datasets, plotting, image
# from nilearn.maskers import NiftiMapsMasker
# from sklearn.decomposition import FastICA
# import pandas as pd
# import tarfile
# import gzip
# #sim required
# import random
# import pingouin as pg
# import seaborn as sns
# #GNN required
# import torch
# import torch.nn.functional as F
# from torch.nn import Linear
# import torch.nn as nn
# import torch_geometric
# from torch_geometric.nn import GCNConv
# from torch_geometric.nn import SAGEConv
# from torch_geometric.nn import GraphConv
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader
# from torch.utils.data import SubsetRandomSampler
# from sklearn.metrics import accuracy_score

# STEP 1. Input Data Selection

**Deliverables:**  
**Goal**: Create different sub-graphs two decrease training times and aid in overall interpretability of GNN performance

**Sub-graph Types**
1. Significant Regions [alpha] 
2. Significant Regions [bonferroni-corrected] 
3. Graph Sampling [GRAPHSaint]

In [2]:
# imports for this section
from ast import literal_eval
import re
import gzip
import pandas as pd
import numpy as np

In [4]:
data_file = 'Data/data_clean.csv.gz'

with gzip.open(data_file) as filepath:
    data = pd.read_csv(filepath, index_col = 'subject_id', 
            converters = {'netmat' : lambda x : np.array(literal_eval(re.sub('(?<!\[)\s+|[\\n]', ', ', x)))})
    
data.head()

,Gender,Age,netmat
subject_id,,,
100206,M,26-30,"[[0.0, 0.61676, 9.5727, -5.4959, 0.34639, 3.00..."
100307,F,26-30,"[[0.0, -0.29664, 17.317, -9.0467, -0.28723, 1...."
100408,M,31-35,"[[0.0, 1.6486, 6.6189, -8.8877, 1.4337, 1.006,..."
100610,M,26-30,"[[0.0, -0.90275, 7.7215, -8.3907, 3.3144, 2.93..."
101006,F,31-35,"[[0.0, -0.088768, 9.4979, -10.412, 1.0646, 4.3..."


In [32]:
sig_edges = np.genfromtxt('significant_edges.csv', delimiter=',')

In [33]:
# full dataset
graphs = np.stack(data.netmat.to_numpy()) # all 1003 correlation matrices
# # significant edges
# graphs_sig = graphs[:, :, []]
# # bonferroni significant edges
# graphs_sig_b = 
# sig_edges = np.genfromtxt('significant_edges.csv', delimiter=',')

In [20]:
# Example: Create a NumPy array of nested NumPy arrays with nested lists
nested_arrays = np.array([
    np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    np.array([[10, 11, 12], [13, 14, 15], [16, 17, 18]]),
    np.array([[19, 20, 21], [22, 23, 24], [25, 26, 27]])
])

# Indexing the lists for each nested NumPy array
first_list = nested_arrays[:, :, 0]
second_list = nested_arrays[:, :, 1]
third_list = nested_arrays[:, :, 2]

print("First lists:")
print(first_list)
print("\nSecond lists:")
print(second_list)
print("\nThird lists:")
print(third_list)

First lists:
[[ 1  4  7]
 [10 13 16]
 [19 22 25]]

Second lists:
[[ 2  5  8]
 [11 14 17]
 [20 23 26]]

Third lists:
[[ 3  6  9]
 [12 15 18]
 [21 24 27]]


In [ ]:
# STEPS:

# STEP 1: INPUT DATA TYPES
# -> FILTERING
# 1. Sig Regions [alpha] 
# 2. Sig Regions [bonferroni-corrected] 
# 3. Graph Sampling [GRAPHSaint]

In [3]:
# STEP 2: MODEL DEVELOPMENT - VIEW EMBEDDING SPACES BEFORE TRAINING
# STEP 3: MODEL SELECTION/FEATURE SELECTION
# we will test many different combinations of inputs and architectures
# INPUT TYPES
# 1. 10 regions [450 male subjects, 550 female subjects (1000 total netmats)]
# 2. 20 regions [450 male subjects, 550 female subjects (1000 total netmats)]
# 3. 50 regions [450 male subjects, 550 female subjects (1000 total netmats)]
# 4. 100 regions [450 male subjects, 550 female subjects (1000 total netmats)]
#
# ARCHITECTURE TYPES:
# -> layers
# 1. 1-layer
# 2. 2-layer
# 3. 3-layer
# 4. 4-layer
# -> squishificaton function (mapping to [-1,1])
# 1. tanh
# 2. ReLu
# 3. sigmoid
# -> embedding space dimension:
# 1. 50
# 2. 100
# 3. 200